## Accessory functions

In [1]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data

In [2]:
def dump_dict(data, name):
    with open(name, 'w', encoding="utf8") as f:
        json.dump(data, f)

# Creating Vocab

In [8]:
train_100 = getJsonData('jsons/train_100.json')
test_100 = getJsonData('jsons/test_100.json')

In [9]:
len(train_100)

55467

In [11]:
print(train_100[0].keys())

# A function that returns the 'year' value:
def myFunc(e):
    return e['int_date']

def temp_print(eclipse_train):
    for i in range(100,110):
        print(eclipse_train[i]['int_date'])

eclipse_train = []

sample_count = 0
for i in range(0,len(train_100)):
    if train_100[i]['project_name'] == 'googlereview_':
        sample_count += 1
        eclipse_train.append(train_100[i])
        #print(train_100[i]['int_date'])
        
print(sample_count)

eclipse_test = []
sample_count = 0
for i in range(0,len(test_100)):
    if test_100[i]['project_name'] == 'googlereview_':
        sample_count += 1
        eclipse_test.append(test_100[i])
        #print(train_100[i]['int_date'])
        
print(sample_count)
#temp_print(eclipse_train)
eclipse_train.sort(key=myFunc)
eclipse_test.sort(key=myFunc)
#print("after_sort")

print(len(eclipse_train),len(eclipse_test))
#temp_print(eclipse_train)
dump_dict(eclipse_train, "project_wise_jsons/googlereview_train.json")
dump_dict(eclipse_test, "project_wise_jsons/googlereview_test.json")


dump_dict(eclipse_train[0:5000], "project_wise_jsons/googlereview_train_first_5000.json")
dump_dict(eclipse_train[5000:5500], "project_wise_jsons/googlereview_test_5500.json")

dump_dict(eclipse_train[0:10000], "project_wise_jsons/googlereview_train_first_10000.json")
dump_dict(eclipse_train[10000:10500], "project_wise_jsons/googlereview_test_10500.json")

dict_keys(['status', 'message', 'comment_id', 'target', 'code_snippet', 'prime_var_dic', 'class_list', 'func_list', 'tokenized_code_snippet', 'tokenized_target', 'tokenized_comment', 'global_index', 'base_code_line_number', 'base_patch_number', 'changed_patch_number', 'code_file_name', 'line_change', 'written_on', 'project_name', 'int_date'])
13907
734
13907 734


In [13]:
largest_train = eclipse_train + eclipse_test[0:93]
largest_test = eclipse_test[93:]


print(len(largest_train), len(largest_test))

dump_dict(largest_train, "project_wise_jsons/googlereview_train_first_14000.json")
dump_dict(largest_test, "project_wise_jsons/googlereview_test_14641.json")

14000 641


In [61]:
train_100 = getJsonData('project_wise_jsons/googlereview_train_first_14000.json')
test_100 = getJsonData('project_wise_jsons/googlereview_test_14641.json')

In [62]:
code_freq = {}
comment_freq = {}

In [63]:
for i in range(len(train_100)):
    source_tokens = train_100[i]['tokenized_code_snippet']
    target_tokens = train_100[i]['tokenized_target']
    comment_tokens = train_100[i]['tokenized_comment']
    code_tokens = source_tokens + target_tokens
    
    for x in code_tokens:
        if x not in code_freq:
            code_freq[x] = 0
        code_freq[x] += 1
        
    for x in comment_tokens:
        if x not in comment_freq:
            comment_freq[x] = 0
        comment_freq[x] += 1

In [64]:
sorted_code_freq = {k: v for k, v in sorted(code_freq.items(), key=lambda item: item[1], reverse=True)}

In [65]:
sorted_comment_freq = {k: v for k, v in sorted(comment_freq.items(), key=lambda item: item[1], reverse=True)}

**Dump the file for future reference**

In [66]:
dump_dict(sorted_code_freq, 'project_wise_jsons/sorted_code_freq_googlereview.json')

In [67]:
dump_dict(sorted_comment_freq, 'project_wise_jsons/sorted_comment_freq_googlereview.json')

**Saving vocab text file**

In [68]:
target_out = open("googlereview_all/14000/target_vocab_10000.txt", "w", encoding='utf-8')
source_out = open("googlereview_all/14000/source_vocab_10000.txt", "w", encoding='utf-8')

In [69]:
## Take the first 'num_code_token' from the codes.

num_code_token = 2000
num_total_token = 10000

for x in list(sorted_code_freq)[:num_code_token]:
    target_out.write(x)
    target_out.write("\n")
target_out.close()

In [70]:
src_vocab = list(sorted_code_freq)[:num_code_token]
for tok in sorted_comment_freq:
    if tok not in src_vocab and len(src_vocab) < num_total_token:
        src_vocab.append(tok)

for x in src_vocab:
    source_out.write(x)
    source_out.write("\n")
source_out.close()

# Creating Training Data

In [71]:
train_100 = getJsonData('project_wise_jsons/googlereview_train_first_14000.json')
test_100 = getJsonData('project_wise_jsons/googlereview_test_14641.json')

## Code+Comment

Write the train_100 file into OpenNMT training data. First CC, then C

In [72]:
src_train = open("googlereview_all/14000/training_data/cc/src-train.txt", 'w')
src_test = open("googlereview_all/14000/training_data/cc/src-test.txt", 'w')
tgt_train = open("googlereview_all/14000/training_data/cc/tgt-train.txt", 'w')
tgt_test = open("googlereview_all/14000/training_data/cc/tgt-test.txt", 'w')

In [73]:
for x in train_100:
    comment = '<|startcomment|> ' + ' '.join(x['tokenized_comment'][:200]) + ' <|endcomment|> '
    code_snippet = ' '.join(x['tokenized_code_snippet']) + '\n'
    target = ' '.join(x['tokenized_target']) + '\n'
    src_train.write(comment)
    src_train.write(code_snippet)
    tgt_train.write(target)
src_train.close()
tgt_train.close()

In [74]:
for x in test_100:
    comment = '<|startcomment|> ' + ' '.join(x['tokenized_comment'][:200]) + ' <|endcomment|> '
    code_snippet = ' '.join(x['tokenized_code_snippet']) + '\n'
    target = ' '.join(x['tokenized_target']) + '\n'
    src_test.write(comment)
    src_test.write(code_snippet)
    tgt_test.write(target)
src_test.close()
tgt_test.close()